# **Data Science w Pythonie**
## Projekt I
### Predykcja wyścigu Grand Prix Emilii-Romanii Formuły 1


In [ ]:
## Import bibliotek

import requests
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

Wybór modelu ARIMA (Autoregressive Integrated Moving Average) do prognozowania miejsc zajętych przez kierowców w Formule 1 ma swoje uzasadnienie z kilku powodów:

Sezonowy charakter danych: W sporcie takim jak Formuła 1, dane mają często wyraźny charakter sezonowy, gdzie każdy sezon składa się z określonej liczby wyścigów. Model ARIMA jest odpowiedni do modelowania sezonowych danych, ponieważ zawiera składową sezonową, która może uchwycić cykliczne wzorce w danych, takie jak sezonowość w wynikach kierowców.

Trendy w danych: ARIMA ma wbudowaną zdolność do uwzględniania trendów w danych. W Formule 1 kierowcy mogą wykazywać tendencje do poprawy lub pogorszenia swoich wyników w ciągu sezonu, co może być uwzględnione przez model ARIMA.

Modelowanie szeregów czasowych: ARIMA jest jednym z najczęściej stosowanych narzędzi do modelowania szeregów czasowych. Dane dotyczące Formuły 1, takie jak wyniki kierowców w kolejnych wyścigach, są naturalnie przedstawiane jako szeregi czasowe, gdzie obserwacje są zależne od czasu.

Prostota interpretacji: ARIMA jest stosunkowo prostym modelem, który może być stosunkowo łatwy do zrozumienia i interpretacji. Może to być ważne, gdy chcemy, aby wyniki modelu były zrozumiałe dla osób spoza dziedziny statystyki lub analizy danych.

Dostępność implementacji: Model ARIMA jest powszechnie dostępny w wielu bibliotekach statystycznych i językach programowania, co ułatwia jego implementację i zastosowanie w praktyce.

Biorąc pod uwagę powyższe czynniki oraz charakter danych związanych z Formułą 1, model ARIMA wydaje się odpowiednim wyborem do prognozowania miejsc zajętych przez kierowców. Jednakże warto również rozważyć inne modele, takie jak modele oparte na sieciach neuronowych, szczególnie gdy dane są bardziej złożone lub zawierają nieliniowe zależności.

W modelu ARIMA możemy dostosowywać parametry (p, d, q):

p (AR - AutoRegressive): Określa stopień autoregresji, czyli liczbę poprzednich obserwacji, które zostaną uwzględnione w modelu. W skrócie, AR oznacza ilość autoregresyjnych kroków w modelu. Na przykład p=5 oznacza, że model będzie uwzględniał ostatnie 5 kroków czasowych.

d (I - Integrated): Określa stopień różnicowania, czyli liczbę różnic, które zostaną zastosowane do szeregu czasowego w celu uczynienia go stacjonarnym. Różnicowanie polega na odejmowaniu obserwacji od ich poprzedników w celu usunięcia trendów lub sezonowości. Zazwyczaj dla danych czasowych d=1 jest wystarczające.

q (MA - Moving Average): Określa stopień średniej ruchomej, czyli liczbę poprzednich błędów prognozy, które zostaną uwzględnione w modelu. MA oznacza ilość średnich ruchomych w modelu. Na przykład q=0 oznacza brak średniej ruchomej w modelu.

W związku z tym (5, 1, 0) oznacza, że model ARIMA zawiera 5 kroków autoregresji (p=5), zastosowano jedno różnicowanie (d=1) oraz brak kroków średniej ruchomej (q=0). Te wartości parametrów mogą być dostosowywane i optymalizowane w zależności od charakterystyki danych i potrzeb modelu.

In [ ]:
## Tworzenie funkcji do pobierania danych z API zawierajacego dane o wszystkich wyscigach F1

def pobierz_dane_api(rocznik, numer_wyscigu):
    url = f"https://ergast.com/api/f1/{rocznik}/{numer_wyscigu}/results.json"
    odpowiedz = requests.get(url)
    dane = odpowiedz.json()
    return dane

In [ ]:
## Pobieranie danych z kolejnych lat wyscigow

wyscigi_2020 = {}
for numer_wyscigu in range(1, 18):
        wyscigi_2020[numer_wyscigu] = pobierz_dane_api(2020, numer_wyscigu)
        
wyscigi_2021 = {}
for numer_wyscigu in range(1, 12):
        wyscigi_2021[numer_wyscigu] = pobierz_dane_api(2021, numer_wyscigu)
        
wyscigi_2022 = {}
for numer_wyscigu in range(1, 23):
        wyscigi_2022[numer_wyscigu] = pobierz_dane_api(2022, numer_wyscigu)
        
wyscigi_2023 = {}
for numer_wyscigu in range(1, 23):
        wyscigi_2023[numer_wyscigu] = pobierz_dane_api(2023, numer_wyscigu)
        
wyscigi_2024 = {}
for numer_wyscigu in range(1, 7):
        wyscigi_2024[numer_wyscigu] = pobierz_dane_api(2024, numer_wyscigu)

In [ ]:
## Przenoszenie danych do DataFrame

rows_2020 = []
for numer_wyscigu in wyscigi_2020.keys():
        data = wyscigi_2020[numer_wyscigu]['MRData']['RaceTable']['Races'][0]
        Year = '2020'
        RaceId = numer_wyscigu
        CircuitId = data['Circuit']['circuitId']
        for results in data['Results']:
            dict1 = {}
            if results["status"] == "Finished":
                position = results["position"]
                points = results["points"]
                DriverId = results['Driver']['driverId']
            
                dict1.update({'Year': Year, 'RaceId': RaceId, 'CircuitId': CircuitId, "position": position, "points": points, 'DriverId': DriverId})
                rows_2020.append(dict1)
df_2020 = pd.DataFrame(rows_2020, columns=["Year", "RaceId", "CircuitId", "position", "points", "DriverId"])

rows_2021 = []
for numer_wyscigu in wyscigi_2021.keys():
        data = wyscigi_2021[numer_wyscigu]['MRData']['RaceTable']['Races'][0]
        Year = '2021'
        RaceId = numer_wyscigu
        CircuitId = data['Circuit']['circuitId']
        for results in data['Results']:
            dict1 = {}
            if results["status"] == "Finished":
                position = results["position"]
                points = results["points"]
                DriverId = results['Driver']['driverId']
            
                dict1.update({'Year': Year, 'RaceId': RaceId, 'CircuitId': CircuitId, "position": position, "points": points, 'DriverId': DriverId})
                rows_2021.append(dict1)
df_2021 = pd.DataFrame(rows_2021, columns=["Year", "RaceId", "CircuitId", "position", "points", "DriverId"])

rows_2022 = []
for numer_wyscigu in wyscigi_2022.keys():
        data = wyscigi_2022[numer_wyscigu]['MRData']['RaceTable']['Races'][0]
        Year = '2022'
        RaceId = numer_wyscigu
        CircuitId = data['Circuit']['circuitId']
        for results in data['Results']:
            dict1 = {}
            if results["status"] == "Finished":
                position = results["position"]
                points = results["points"]
                DriverId = results['Driver']['driverId']
            
                dict1.update({'Year': Year, 'RaceId': RaceId, 'CircuitId': CircuitId, "position": position, "points": points, 'DriverId': DriverId})
                rows_2022.append(dict1)
df_2022 = pd.DataFrame(rows_2022, columns=["Year", "RaceId", "CircuitId", "position", "points", "DriverId"])

rows_2023 = []
for numer_wyscigu in wyscigi_2023.keys():
        data = wyscigi_2023[numer_wyscigu]['MRData']['RaceTable']['Races'][0]
        Year = '2023'
        RaceId = numer_wyscigu
        CircuitId = data['Circuit']['circuitId']
        for results in data['Results']:
            dict1 = {}
            if results["status"] == "Finished":
                position = results["position"]
                points = results["points"]
                DriverId = results['Driver']['driverId']
            
                dict1.update({'Year': Year, 'RaceId': RaceId, 'CircuitId': CircuitId, "position": position, "points": points, 'DriverId': DriverId})
                rows_2023.append(dict1)
df_2023 = pd.DataFrame(rows_2023, columns=["Year", "RaceId", "CircuitId", "position", "points", "DriverId"])

rows_2024 = []
for numer_wyscigu in wyscigi_2024.keys():
        data = wyscigi_2024[numer_wyscigu]['MRData']['RaceTable']['Races'][0]
        Year = '2024'
        RaceId = numer_wyscigu
        CircuitId = data['Circuit']['circuitId']
        for results in data['Results']:
            dict1 = {}
            if results["status"] == "Finished":
                position = results["position"]
                points = results["points"]
                DriverId = results['Driver']['driverId']
            
                dict1.update({'Year': Year, 'RaceId': RaceId, 'CircuitId': CircuitId, "position": position, "points": points, 'DriverId': DriverId})
                rows_2024.append(dict1)
df_2024 = pd.DataFrame(rows_2024, columns=["Year", "RaceId", "CircuitId", "position", "points", "DriverId"])

df = pd.concat([df_2020, df_2021, df_2022, df_2023, df_2024], ignore_index=True)
del df_2020, df_2021, df_2022, df_2023, df_2024

In [ ]:
# Cast
df['position'] = pd.to_numeric(df['position'])
df['points'] = pd.to_numeric(df['points'])
df['Year'] = pd.to_numeric(df['Year'])

In [ ]:
##Implementacja modelu ARIMA

##tworzenie "klucza", na którego podstawie będzie przebiegała prognoza
df['wyscig_sezon'] = df['Year'].astype(str) + '-' + df['RaceId'].astype(str)

##tworzenie obiektu drivers, który zawiera unikalnych kierowców
drivers = df['DriverId'].unique()

##Słownik na przyszłe wyniki
predictions = {}

##Prognozowanie punktów dla każdego kierowcy
for driver in drivers:
    driver_data = df[df['DriverId'] == driver].set_index('wyscig_sezon')['points']
    
    # Sprawdzamy, czy mamy wystarczająco dużo danych do modelowania
    if len(driver_data) > 5:  #Minimum 6 punktów danych do ARIMA(5,1,0)
        try:
            ##Dopasowanie modelu ARIMA (p, d, q)
            model = ARIMA(driver_data, order=(10, 1, 0))
            model_fit = model.fit()

            # Prognoza punktów na kolejny wyścig
            forecast = model_fit.forecast(steps=1)
            predictions[driver] = forecast.iloc[0]
        except Exception as e:
            print(f"Nie udało się dopasować modelu dla kierowcy {driver}: {e}")
    #else:
        #print(f"Zbyt mało danych dla kierowcy {driver} do modelowania ARIMA")

# Sortowanie wyników według prognozowanych punktów
sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)

# Wyświetlenie top 5 kierowców
top_5_drivers = sorted_predictions[:5]    

In [ ]:
## Output 
print("Prognoza pierwszej piątki kierowców:")
for i, (driver, points) in enumerate(top_5_drivers, start=1):
    print(f"Miejsce {i}: Kierowca {driver} z prognozowanymi punktami {points:.2f}")